In [ ]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import codecs

Input file

In [ ]:
input_file='data_tweets'
training_file = input_file+'.txt'
text = codecs.open(training_file, 'r',"utf-8").read()

Network parameters

In [ ]:
maxlen = 40
step = 1
nb_dvd=10
learn_rate=0.001
batch_size=len(text)//nb_dvd
nb_iter=100

nb_units=128
epochs=batch_size//maxlen
epochs=10

Vocabulary creation (word level)

In [ ]:
word=text.split(" ")
char_indices = dict((c, i) for i, c in enumerate(word))
indices_char = dict((i, c) for i, c in enumerate(word))

Data processing to feed the network

In [ ]:
def generate_arrays_from_file(text,iteration):
    #while True:
        #f = open(path)
        #text = codecs.open(path, 'r',"utf-8")
        #for i in range(batch_size):
            # cut the text in semi-redundant sequences of maxlen characters
    index= iteration
    textl=word[index:index+batch_size]       
    sentences = []
    next_chars = []
    for j in range(1,batch_size - maxlen-1, step):
        sentences.append(textl[j: j + maxlen])
        next_chars.append(textl[j + maxlen])
    #print('nb sequences:', len(sentences))
    print('Vectorization...')
    X = np.zeros((len(sentences), maxlen, len(word)), dtype=np.bool)
    y = np.zeros((len(sentences), len(word)), dtype=np.bool)
    for k, sentence in enumerate(sentences):
        for t, words in enumerate(sentence):
            X[k, t, char_indices[str(words)]] = 1
            y[k, char_indices[next_chars[k]]] = 1
    return X, y
 

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Building the network

In [ ]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(nb_units, input_shape=(maxlen, len(word))))
model.add(Dense(len(word)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=learn_rate)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Training

In [ ]:
%%time

# train the model, output generated text after each iteration
for iteration in range(1, nb_iter):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    X,y=generate_arrays_from_file(text,iteration)
    model.fit(X,y,batch_size=batch_size, epochs=epochs)



Saving the model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")